In [1]:
import tensorflow as tf
import keras as keras
import keras.datasets.cifar10 as cifar10
from keras.layers import Input, MaxPool2D, Conv2D, Flatten, Dense
from keras.layers import concatenate
from keras import optimizers

2023-10-16 22:20:45.831128: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-16 22:20:45.859922: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-16 22:20:45.859949: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-16 22:20:45.859977: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-16 22:20:45.865652: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-16 22:20:45.866125: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [3]:
from keras import backend as K
K.set_image_data_format("channels_last")

In [4]:
X_train = X_train.reshape(X_train.shape[0], 32, 32, 3).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 32, 32, 3).astype('float32')
X_train = X_train/255.0
X_test = X_test/255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes = num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes = num_classes)

In [5]:
input_img = Input(shape=(32, 32, 3))
tower_1 = Conv2D(64, (1,1),padding="same", activation="relu")(input_img)
tower_1 = Conv2D(64, (3,3), padding="same", activation="relu")(tower_1)


tower_2 = Conv2D(64, (1,1),padding="same", activation="relu")(input_img)
tower_2 = Conv2D(64, (5,5), padding="same", activation="relu")(tower_2)


tower_3 = MaxPool2D((3,3),strides=1, padding="same")(input_img)
tower_3 = Conv2D(64, (1,1), padding="same", activation="relu")(tower_3)

2023-10-16 22:20:49.002525: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-10-16 22:20:49.002554: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: gabriel-Dell-G15-5525
2023-10-16 22:20:49.002566: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: gabriel-Dell-G15-5525
2023-10-16 22:20:49.002721: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.113.1
2023-10-16 22:20:49.002757: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.113.1
2023-10-16 22:20:49.002767: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.113.1


In [6]:
output = concatenate([tower_1, tower_2, tower_3], axis = 3)
output = Flatten()(output)
out = Dense(10, activation = "softmax")(output)

In [7]:
from keras.models import Model
model  = Model(inputs=input_img, outputs = out)
epochs = 5
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(), metrics=[tf.metrics.Precision()])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 32, 32, 64)           256       ['input_1[0][0]']             
                                                                                                  
 conv2d_2 (Conv2D)           (None, 32, 32, 64)           256       ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 32, 32, 3)            0         ['input_1[0][0]']             
 D)                                                                                           

In [8]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = epochs, batch_size = 32)

Epoch 1/5


2023-10-16 22:20:49.387086: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 614400000 exceeds 10% of free system memory.


1563/1563 [==============================] - 156s 99ms/step - loss: 1.5951 - precision: 0.6605 - val_loss: 1.3479 - val_precision: 0.7113
Epoch 2/5
1563/1563 [==============================] - 155s 99ms/step - loss: 1.2429 - precision: 0.7270 - val_loss: 1.3162 - val_precision: 0.6867
Epoch 3/5
1563/1563 [==============================] - 155s 99ms/step - loss: 1.0954 - precision: 0.7551 - val_loss: 1.1887 - val_precision: 0.7076
Epoch 4/5
1563/1563 [==============================] - 152s 97ms/step - loss: 0.9833 - precision: 0.7788 - val_loss: 1.2226 - val_precision: 0.6932
Epoch 5/5
1563/1563 [==============================] - 151s 96ms/step - loss: 0.8806 - precision: 0.7957 - val_loss: 1.2091 - val_precision: 0.6970
